In [111]:
import pandas as pd
import sys
import os
import numpy as np
!pip install plotly_express
!pip install nbformat
!pip install -U kaleido
!pip isntall plotly
import plotly_express as px
import plotly.graph_objects as go
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))

ERROR: unknown command "isntall" - maybe you meant "install"


In [112]:
from scipy import stats

In [113]:
def get_stats(method, model, data=None):
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'
    df = pd.read_csv(path)
    res_cind = df.groupby(['dataset'])['cindex_test'].agg([np.mean, np.std]).rename(
    columns={
        'mean': 'mean_cindex',
        'std': 'std_cindex'
    })
    res_ibs = df.groupby(['dataset'])['ibs_test'].agg([np.mean, np.std]).rename(
    columns={
        'mean': 'mean_ibs',
        'std': 'std_ibs'
    })
    df_res = pd.concat([res_cind,res_ibs],axis=1).reset_index()
    df_res['model'] = model
    return df_res

def get_boxplot_cindex( model, title, data=None):
    method = 'dl'
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'

    df_dl = pd.read_csv(path)
    method = 'gbdt'
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'
    df_gbdt = pd.read_csv(path)

    fig = go.Figure()
    fig.add_trace(go.Box( y=df_dl['cindex_test'],x=df_dl['dataset'],name="DL"))
    fig.add_trace(go.Box( y=df_gbdt['cindex_test'],x=df_dl['dataset'], name="GBDT"))
    fig.update_layout(title_text=title, boxmode='group')
    #fig = px.box(df, x='dataset', y='cindex_test', title=title, labels={'cindex_test':'c-index score'})
    if data:
        fig.write_image("images/"+model+"_cindex_"+data+".png", scale=5)
    else:
        fig.write_image("images/"+model+"_cindex.png", scale=5)
    return fig

def get_boxplot_ibs( model, title, data=None):
    method = 'dl'
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'

    df_dl = pd.read_csv(path)
    method = 'gbdt'
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'
    df_gbdt = pd.read_csv(path)
    print(df_gbdt.columns)
    fig = go.Figure()
    fig.add_trace(go.Box( y=df_dl['ibs_test'],x=df_dl['dataset'],name="DL"))
    fig.add_trace(go.Box( y=df_gbdt['ibs_test'],x=df_dl['dataset'], name="GBDT"))
    fig.update_layout(title_text=title, boxmode='group')
    #fig = px.box(df, x='dataset', y='cindex_test', title=title, labels={'cindex_test':'c-index score'})
    if data:
        fig.write_image("images/"+model+"_ibs_"+data+".png", scale=5)
    else:
        fig.write_image("images/"+model+"_ibs.png", scale=5)

    return fig

def get_wttest(model, data=None):
    method = 'dl'
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'

    df_dl = pd.read_csv(path)

    method = 'gbdt'
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'

    df_gbdt = pd.read_csv(path)

    # Perform the Wilcoxon signed-rank test
    for dataset in df_dl['dataset'].unique().tolist():
        dfgb = df_gbdt[df_gbdt['dataset']==dataset]
        dfdl = df_dl[df_dl['dataset']==dataset]
        statistic, p_value = stats.wilcoxon(dfgb['cindex_test'], dfdl['cindex_test'])
        print(statistic, p_value)

    return statistic, p_value



## Breslow Stats

In [114]:
#df_gbdt = get_stats(method='gbdt', model='breslow', data='tcga')
df_dl = get_stats(method='dl', model='breslow', data=None)
get_wttest(model='breslow', data=None)
get_wttest(model='breslow', data='tcga')

4.0 0.4375
4.0 0.4375
0.0 0.0625
0.0 0.0625
1.0 0.125
7.0 1.0
7.0 1.0
3.0 0.3125
0.0 0.0625
1.0 0.125
7.0 1.0
5.0 0.625
3.0 0.3125
7.0 1.0


(7.0, 1.0)

In [115]:
get_boxplot_cindex( model='breslow', title='C-index: Breslow GBDT vs DL', data=None)

In [116]:
get_boxplot_cindex( model='breslow', title='C-index: Breslow GBDT vs DL', data='tcga')

In [117]:
get_boxplot_ibs( model='breslow', title='IBS: Breslow GBDT vs DL', data=None)

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


In [118]:
get_boxplot_ibs( model='breslow', title='IBS: Breslow GBDT vs DL', data='tcga')

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


## Efron Stats

In [119]:
get_boxplot_cindex( model='efron', title='C-index: Efron GBDT vs DL', data=None)

In [120]:
get_boxplot_cindex( model='efron', title='C-index: Efron GBDT vs DL', data='tcga')

In [121]:
get_boxplot_ibs( model='efron', title='IBS: Efron GBDT vs DL', data=None)

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


In [122]:
get_boxplot_ibs( model='efron', title='IBS: Efron GBDT vs DL', data='tcga')

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


## AFT Stats

In [123]:
get_boxplot_cindex( model='aft', title='C-index: AFT GBDT vs DL', data=None)

In [124]:
get_boxplot_cindex( model='aft', title='C-index: AFT GBDT vs DL', data='tcga')

In [125]:
get_boxplot_ibs( model='aft', title='IBS: AFT GBDT vs DL', data=None)

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


In [126]:
get_boxplot_ibs( model='aft', title='IBS: AFT GBDT vs DL', data='tcga')

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


## AH

In [127]:
get_boxplot_cindex( model='ah', title='C-index: AH GBDT vs DL', data=None)

In [128]:
get_boxplot_cindex( model='ah', title='C-index: AH GBDT vs DL', data='tcga')

In [129]:
get_boxplot_ibs( model='ah', title='IBS: AH GBDT vs DL', data=None)

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


In [130]:
get_boxplot_ibs( model='ah', title='IBS: AH GBDT vs DL', data='tcga')

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


## EH

In [131]:
get_boxplot_cindex( model='eh', title='C-index: EH GBDT vs DL', data=None)

In [132]:
get_boxplot_cindex( model='eh', title='C-index: EH GBDT vs DL', data='tcga')

In [133]:
get_boxplot_ibs( model='eh', title='IBS: EH GBDT vs DL', data=None)

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


In [134]:
get_boxplot_ibs( model='eh', title='IBS: EH GBDT vs DL', data='tcga')

Index(['model', 'dataset', 'cindex_train', 'cindex_test', 'ibs_train',
       'ibs_test'],
      dtype='object')


In [135]:
## C-boosting

## Comparison Table

### Not TCGA Data C-index

In [160]:
df_gbdt_breslow = get_stats(method='gbdt', model='breslow', data=None)
df_dl_breslow = get_stats(method='dl', model='breslow', data=None)
df_gbdt_efron = get_stats(method='gbdt', model='efron', data=None)
df_dl_efron = get_stats(method='dl', model='efron', data=None)
df_gbdt_aft = get_stats(method='gbdt', model='aft', data=None)
df_dl_aft = get_stats(method='dl', model='aft', data=None)
df_gbdt_ah = get_stats(method='gbdt', model='ah', data=None)
df_dl_ah = get_stats(method='dl', model='ah', data=None)
df_gbdt_eh = get_stats(method='gbdt', model='eh', data=None)
df_dl_eh = get_stats(method='dl', model='eh', data=None)


In [161]:
df_gbdt = pd.concat([df_gbdt_breslow,df_gbdt_efron,df_gbdt_aft, df_gbdt_ah,df_gbdt_eh], axis=0)
df_dl = pd.concat([df_dl_breslow,df_dl_efron,df_dl_aft,df_dl_ah, df_dl_eh], axis=0)

In [162]:
df_final = df_gbdt[['dataset','model']]
df_final['comparison'] = df_gbdt['mean_cindex']/df_dl['mean_cindex']
df_final = df_final.pivot(index='model', columns='dataset', values='comparison').round(4)
df_final

/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/2961948130.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



dataset,FLCHAIN,METABRIC,RGBSG,SUPPORT
model,,,,
aft,1.0147,1.0364,1.0231,1.1160
ah,1.0868,1.0006,0.9900,1.0023
breslow,0.9864,1.0335,1.0593,1.3520
efron,0.9911,1.0077,1.0284,1.0865
eh,0.9993,1.0284,1.0304,1.1082


In [163]:
# Define a function to color cells
def color_cells(val):
    color = 'gray5' if val > 1 else 'gray3' if val < 1 else 'white'
    return f'\\cellcolor{{{color}}} {val}'

# Apply color_cells to df and convert to LaTeX
df = df_final.applymap(color_cells)
latex = df.to_latex(escape=False)

print(latex)


\begin{tabular}{lllll}
\toprule
dataset &                   FLCHAIN &                  METABRIC &                     RGBSG &                   SUPPORT \\
model   &                           &                           &                           &                           \\
\midrule
aft     &  \cellcolor{gray5} 1.0147 &  \cellcolor{gray5} 1.0364 &  \cellcolor{gray5} 1.0231 &   \cellcolor{gray5} 1.116 \\
ah      &  \cellcolor{gray5} 1.0868 &  \cellcolor{gray5} 1.0006 &    \cellcolor{gray3} 0.99 &  \cellcolor{gray5} 1.0023 \\
breslow &  \cellcolor{gray3} 0.9864 &  \cellcolor{gray5} 1.0335 &  \cellcolor{gray5} 1.0593 &   \cellcolor{gray5} 1.352 \\
efron   &  \cellcolor{gray3} 0.9911 &  \cellcolor{gray5} 1.0077 &  \cellcolor{gray5} 1.0284 &  \cellcolor{gray5} 1.0865 \\
eh      &  \cellcolor{gray3} 0.9993 &  \cellcolor{gray5} 1.0284 &  \cellcolor{gray5} 1.0304 &  \cellcolor{gray5} 1.1082 \\
\bottomrule
\end{tabular}



/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/306435221.py:8: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



### Not TCGA Data IBS

In [164]:
df_gbdt = pd.concat([df_gbdt_breslow,df_gbdt_efron,df_gbdt_aft, df_gbdt_ah,df_gbdt_eh], axis=0)
df_dl = pd.concat([df_dl_breslow,df_dl_efron,df_dl_aft,df_dl_ah, df_dl_eh], axis=0)

In [141]:
df_final = df_gbdt[['dataset','model']]
df_final['comparison'] = df_gbdt['mean_ibs']/df_dl['mean_ibs']
df_final = df_final.pivot(index='model', columns='dataset', values='comparison').round(4)
df_final

/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/476621603.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



dataset,FLCHAIN,METABRIC,RGBSG,SUPPORT
model,,,,
aft,0.9557,0.9496,0.9750,0.9889
ah,0.9623,1.0054,1.0023,1.0008
breslow,0.9969,0.9440,0.9498,0.6806
efron,0.9857,0.9659,0.9922,0.9598
eh,0.9895,0.9697,0.9793,0.9182


In [142]:
# Define a function to color cells
def color_cells(val):
    color = 'gray3' if val > 1 else 'gray5' if val < 1 else 'white'
    return f'\\cellcolor{{{color}}} {val}'

# Apply color_cells to df and convert to LaTeX
df = df_final.applymap(color_cells)
latex = df.to_latex(escape=False)

print(latex)


\begin{tabular}{lllll}
\toprule
dataset &                   FLCHAIN &                  METABRIC &                     RGBSG &                   SUPPORT \\
model   &                           &                           &                           &                           \\
\midrule
aft     &  \cellcolor{gray5} 0.9557 &  \cellcolor{gray5} 0.9496 &   \cellcolor{gray5} 0.975 &  \cellcolor{gray5} 0.9889 \\
ah      &  \cellcolor{gray5} 0.9623 &  \cellcolor{gray3} 1.0054 &  \cellcolor{gray3} 1.0023 &  \cellcolor{gray3} 1.0008 \\
breslow &  \cellcolor{gray5} 0.9969 &   \cellcolor{gray5} 0.944 &  \cellcolor{gray5} 0.9498 &  \cellcolor{gray5} 0.6806 \\
efron   &  \cellcolor{gray5} 0.9857 &  \cellcolor{gray5} 0.9659 &  \cellcolor{gray5} 0.9922 &  \cellcolor{gray5} 0.9598 \\
eh      &  \cellcolor{gray5} 0.9895 &  \cellcolor{gray5} 0.9697 &  \cellcolor{gray5} 0.9793 &  \cellcolor{gray5} 0.9182 \\
\bottomrule
\end{tabular}



/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/2467697870.py:8: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [143]:
## TCGA Data

In [144]:
df_gbdt_breslow = get_stats(method='gbdt', model='breslow', data='tcga')
df_dl_breslow = get_stats(method='dl', model='breslow', data='tcga')
df_gbdt_efron = get_stats(method='gbdt', model='efron', data='tcga')
df_dl_efron = get_stats(method='dl', model='efron', data='tcga')
df_gbdt_aft = get_stats(method='gbdt', model='aft', data='tcga')
df_dl_aft = get_stats(method='dl', model='aft', data='tcga')
df_gbdt_ah = get_stats(method='gbdt', model='ah', data='tcga')
df_dl_ah = get_stats(method='dl', model='ah', data='tcga')
df_gbdt_eh = get_stats(method='gbdt', model='eh', data='tcga')
df_dl_eh = get_stats(method='dl', model='eh', data='tcga')

In [145]:
df_gbdt = pd.concat([df_gbdt_breslow,df_gbdt_efron,df_gbdt_aft, df_gbdt_ah,df_gbdt_eh], axis=0)
df_dl = pd.concat([df_dl_breslow,df_dl_efron,df_dl_aft,df_dl_ah, df_dl_eh], axis=0)
df_final = df_gbdt[['dataset','model']]
df_final['comparison'] = df_gbdt['mean_cindex']/df_dl['mean_cindex']
df_final = df_final.pivot(index='model', columns='dataset', values='comparison').round(4)
df_final

/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/3591916392.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



dataset,BLCA,BRCA,HNSC,KIRC,LGG,LIHC,LUAD,LUSC,OV,STAD
model,,,,,,,,,,
aft,0.7614,0.9856,0.7341,0.9684,1.0109,0.9694,0.9770,0.5242,0.9662,0.9762
ah,0.8325,1.1980,1.1078,0.9050,0.9880,1.1204,1.0421,1.2880,1.0412,0.9622
breslow,1.0783,0.9571,0.9841,1.0536,1.1527,0.6804,0.9497,0.9619,1.1045,1.0125
efron,1.0987,1.0560,1.0372,0.9137,1.1622,1.0220,1.1056,0.8502,1.0029,1.0596
eh,0.9765,0.9668,0.8726,0.9893,1.0292,0.8278,0.9637,0.9777,1.0273,0.9505


In [146]:
# Define a function to color cells
def color_cells(val):
    color = 'gray5' if val > 1 else 'gray3' if val < 1 else 'white'
    return f'\\cellcolor{{{color}}} {val}'

# Apply color_cells to df and convert to LaTeX
df = df_final.applymap(color_cells)
latex = df.to_latex(escape=False)

print(latex)


\begin{tabular}{lllllllllll}
\toprule
dataset &                      BLCA &                      BRCA &                      HNSC &                      KIRC &                       LGG &                      LIHC &                      LUAD &                      LUSC &                        OV &                      STAD \\
model   &                           &                           &                           &                           &                           &                           &                           &                           &                           &                           \\
\midrule
aft     &  \cellcolor{gray3} 0.7614 &  \cellcolor{gray3} 0.9856 &  \cellcolor{gray3} 0.7341 &  \cellcolor{gray3} 0.9684 &  \cellcolor{gray5} 1.0109 &  \cellcolor{gray3} 0.9694 &   \cellcolor{gray3} 0.977 &  \cellcolor{gray3} 0.5242 &  \cellcolor{gray3} 0.9662 &  \cellcolor{gray3} 0.9762 \\
ah      &  \cellcolor{gray3} 0.8325 &   \cellcolor{gray5} 1.198 &  \cellcolor{gr

/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/306435221.py:8: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [147]:
### TCGA IBS

In [148]:
df_gbdt = pd.concat([df_gbdt_breslow,df_gbdt_efron,df_gbdt_aft, df_gbdt_ah,df_gbdt_eh], axis=0)
df_dl = pd.concat([df_dl_breslow,df_dl_efron,df_dl_aft,df_dl_ah, df_dl_eh], axis=0)
df_final = df_gbdt[['dataset','model']]
df_final['comparison'] = df_gbdt['mean_ibs']/df_dl['mean_ibs']
df_final = df_final.pivot(index='model', columns='dataset', values='comparison').round(4)
df_final

/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/2108415598.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



dataset,BLCA,BRCA,HNSC,KIRC,LGG,LIHC,LUAD,LUSC,OV,STAD
model,,,,,,,,,,
aft,1.0127,0.8880,0.9852,1.0394,0.9475,1.0230,0.9372,1.0220,1.0172,0.9823
ah,1.0052,0.9854,0.9959,0.9810,0.9994,0.9956,1.0978,1.0139,1.0176,0.9986
breslow,0.9752,0.9864,0.7719,0.9636,0.7714,1.0045,0.9720,1.0403,0.9609,0.9900
efron,0.9315,1.0358,1.0362,1.0148,0.8577,0.9334,0.9926,1.0257,1.0890,1.0170
eh,0.9702,0.6791,1.0397,0.9994,0.8346,1.0566,0.9777,0.9950,0.9805,0.9523


In [149]:
# Define a function to color cells
def color_cells(val):
    color = 'gray3' if val > 1 else 'gray5' if val < 1 else 'white'
    return f'\\cellcolor{{{color}}} {val}'

# Apply color_cells to df and convert to LaTeX
df = df_final.applymap(color_cells)
latex = df.to_latex(escape=False)

print(latex)

\begin{tabular}{lllllllllll}
\toprule
dataset &                      BLCA &                      BRCA &                      HNSC &                      KIRC &                       LGG &                      LIHC &                      LUAD &                      LUSC &                        OV &                      STAD \\
model   &                           &                           &                           &                           &                           &                           &                           &                           &                           &                           \\
\midrule
aft     &  \cellcolor{gray3} 1.0127 &   \cellcolor{gray5} 0.888 &  \cellcolor{gray5} 0.9852 &  \cellcolor{gray3} 1.0394 &  \cellcolor{gray5} 0.9475 &   \cellcolor{gray3} 1.023 &  \cellcolor{gray5} 0.9372 &   \cellcolor{gray3} 1.022 &  \cellcolor{gray3} 1.0172 &  \cellcolor{gray5} 0.9823 \\
ah      &  \cellcolor{gray3} 1.0052 &  \cellcolor{gray5} 0.9854 &  \cellcolor{gr

/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_71741/2669649418.py:8: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



## Compare highest performing model

In [165]:
def get_stats(method, model, data=None):
    if data:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_'+data+'_results.csv' 
    else:
        path = one_level_up+'/'+model+'/metrics/'+model+'_'+method+'_results.csv'
    df = pd.read_csv(path)
    res_cind = df.groupby(['dataset'])['cindex_test'].agg([np.mean, np.std]).rename(
    columns={
        'mean': 'mean_cindex',
        'std': 'std_cindex'
    })
    res_ibs = df.groupby(['dataset'])['ibs_test'].agg([np.mean, np.std]).rename(
    columns={
        'mean': 'mean_ibs',
        'std': 'std_ibs'
    })
    df_res = pd.concat([res_cind,res_ibs],axis=1).reset_index()
    df_res['model'] = model
    df_res['method'] = method
    return df_res

In [174]:
df_gbdt_breslow = get_stats(method='gbdt', model='breslow', data=None)
df_dl_breslow = get_stats(method='dl', model='breslow', data=None)
df_gbdt_efron = get_stats(method='gbdt', model='efron', data=None)
df_dl_efron = get_stats(method='dl', model='efron', data=None)
df_gbdt_aft = get_stats(method='gbdt', model='aft', data=None)
df_dl_aft = get_stats(method='dl', model='aft', data=None)
df_gbdt_ah = get_stats(method='gbdt', model='ah', data=None)
df_dl_ah = get_stats(method='dl', model='ah', data=None)
df_gbdt_eh = get_stats(method='gbdt', model='eh', data=None)
df_dl_eh = get_stats(method='dl', model='eh', data=None)

df_all = pd.concat([df_gbdt_breslow,df_gbdt_efron,df_gbdt_aft, df_gbdt_ah,df_gbdt_eh, df_dl_breslow,df_dl_efron,df_dl_aft,df_dl_ah, df_dl_eh], axis=0)
df_all.groupby(['dataset','model','method'])['mean_cindex'].max().round(4) #.sort_values(ascending=False)

dataset   model    method
FLCHAIN   aft      dl        0.7855
                   gbdt      0.7970
          ah       dl        0.5888
                   gbdt      0.6399
          breslow  dl        0.7920
                   gbdt      0.7812
          efron    dl        0.7882
                   gbdt      0.7812
          eh       dl        0.7906
                   gbdt      0.7900
METABRIC  aft      dl        0.6180
                   gbdt      0.6405
          ah       dl        0.5684
                   gbdt      0.5688
          breslow  dl        0.5990
                   gbdt      0.6191
          efron    dl        0.6123
                   gbdt      0.6170
          eh       dl        0.6404
                   gbdt      0.6586
RGBSG     aft      dl        0.6605
                   gbdt      0.6758
          ah       dl        0.5294
                   gbdt      0.5241
          breslow  dl        0.6251
                   gbdt      0.6622
          efron    dl        0.6440
  

In [176]:
df_all.groupby(['dataset','model','method'])['mean_ibs'].min().round(4)

dataset   model    method
FLCHAIN   aft      dl        0.0991
                   gbdt      0.0947
          ah       dl        0.1290
                   gbdt      0.1241
          breslow  dl        0.1009
                   gbdt      0.1006
          efron    dl        0.1020
                   gbdt      0.1006
          eh       dl        0.0965
                   gbdt      0.0955
METABRIC  aft      dl        0.1790
                   gbdt      0.1699
          ah       dl        0.1866
                   gbdt      0.1876
          breslow  dl        0.1786
                   gbdt      0.1686
          efron    dl        0.1748
                   gbdt      0.1689
          eh       dl        0.1718
                   gbdt      0.1666
RGBSG     aft      dl        0.1826
                   gbdt      0.1780
          ah       dl        0.2031
                   gbdt      0.2036
          breslow  dl        0.1942
                   gbdt      0.1845
          efron    dl        0.1859
  

In [178]:
df_gbdt_breslow = get_stats(method='gbdt', model='breslow', data='tcga')
df_dl_breslow = get_stats(method='dl', model='breslow', data='tcga')
df_gbdt_efron = get_stats(method='gbdt', model='efron', data='tcga')
df_dl_efron = get_stats(method='dl', model='efron', data='tcga')
df_gbdt_aft = get_stats(method='gbdt', model='aft', data='tcga')
df_dl_aft = get_stats(method='dl', model='aft', data='tcga')
df_gbdt_ah = get_stats(method='gbdt', model='ah', data='tcga')
df_dl_ah = get_stats(method='dl', model='ah', data='tcga')
df_gbdt_eh = get_stats(method='gbdt', model='eh', data='tcga')
df_dl_eh = get_stats(method='dl', model='eh', data='tcga')

df_all = pd.concat([df_gbdt_breslow,df_gbdt_efron,df_gbdt_aft, df_gbdt_ah,df_gbdt_eh, df_dl_breslow,df_dl_efron,df_dl_aft,df_dl_ah, df_dl_eh], axis=0)


dataset  model    method
BLCA     aft      dl        0.6246
                  gbdt      0.4756
         ah       dl        0.5021
                  gbdt      0.4180
         breslow  dl        0.5068
                             ...  
STAD     breslow  gbdt      0.4940
         efron    dl        0.4902
                  gbdt      0.5194
         eh       dl        0.5518
                  gbdt      0.5244
Name: mean_cindex, Length: 100, dtype: float64

In [180]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_all.groupby(['dataset','model','method'])['mean_cindex'].max().round(4))

dataset  model    method
BLCA     aft      dl        0.6246
                  gbdt      0.4756
         ah       dl        0.5021
                  gbdt      0.4180
         breslow  dl        0.5068
                  gbdt      0.5465
         efron    dl        0.5313
                  gbdt      0.5837
         eh       dl        0.6253
                  gbdt      0.6107
BRCA     aft      dl        0.5844
                  gbdt      0.5759
         ah       dl        0.3324
                  gbdt      0.3983
         breslow  dl        0.4809
                  gbdt      0.4603
         efron    dl        0.4777
                  gbdt      0.5044
         eh       dl        0.5672
                  gbdt      0.5484
HNSC     aft      dl        0.5805
                  gbdt      0.4261
         ah       dl        0.4641
                  gbdt      0.5141
         breslow  dl        0.5676
                  gbdt      0.5586
         efron    dl        0.5194
                  gbdt      0.